In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import utils
import os
%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

from IPython.display import SVG, Image
import tensorflow as tf

# category images

In [2]:
for expression in os.listdir("train/"):
    print(str(len(os.listdir("train/" + expression))) + " " + expression + " images")  
    

3995 angry images
436 disgust images
4097 fear images
7214 happy images
4965 neutral images
4830 sad images
3171 surprise images


# Generate Training and Validation Batches 

In [3]:
img_size = 48
batch_size = 64

datagen_train = ImageDataGenerator(horizontal_flip=True)
train_generator = datagen_train.flow_from_directory("train/",
                                                   target_size = (img_size,img_size),
                                                    color_mode = 'grayscale',
                                                    batch_size = batch_size, 
                                                    class_mode = 'categorical',
                                                    shuffle = True)

datagen_validation = ImageDataGenerator(horizontal_flip=True)
validation_generator = datagen_validation.flow_from_directory("test/",
                                                   target_size = (img_size,img_size),
                                                    color_mode = 'grayscale',
                                                    batch_size = batch_size, 
                                                    class_mode = 'categorical',
                                                    shuffle = True)
                                                    

Found 28708 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


# Create CNN model

In [4]:
model = Sequential()

# 1 - conv layer
model.add(Conv2D(64, (3,3), padding='same', input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# 2 - conv layer
model.add(Conv2D(128, (5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# 3 - conv layer
model.add(Conv2D(512, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# 4 - conv layer
model.add(Conv2D(512, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

# FC 1
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

# FC 2
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(7, activation='softmax'))

In [5]:
opt = Adam(lr = 0.0005)

model.compile(optimizer = opt,
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 64)        256       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 128)       204928    
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 128)       5

# Train and Evaluate Model

In [6]:
epochs = 15
steps_per_epoch = train_generator.n//train_generator.batch_size
validation_steps = validation_generator.n//validation_generator.batch_size

checkpoint = ModelCheckpoint("model_weight.h5",
                            monitor = 'val_accuracy',
                            save_weights_only = True,
                            mode = 'max',
                            verbose=1)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                             factor=0.1,
                             patience=2,
                             min_lr=0.00001,
                             mode='auto')

callbacks = [checkpoint, reduce_lr]

history = model.fit(x=train_generator,
                   steps_per_epoch=steps_per_epoch,
                   epochs=epochs,
                   validation_data=validation_generator,
                   validation_steps=validation_steps,
                   callbacks=callbacks)


Epoch 1/15
448/448 [==============================] - ETA: 0s - loss: 1.7940 - accuracy: 0.3110
Epoch 00001: saving model to model_weight.h5
448/448 [==============================] - 1176s 3s/step - loss: 1.7940 - accuracy: 0.3110 - val_loss: 1.6287 - val_accuracy: 0.3876
Epoch 2/15
448/448 [==============================] - ETA: 0s - loss: 1.4767 - accuracy: 0.4335
Epoch 00002: saving model to model_weight.h5
448/448 [==============================] - 1006s 2s/step - loss: 1.4767 - accuracy: 0.4335 - val_loss: 1.4151 - val_accuracy: 0.4453
Epoch 3/15
448/448 [==============================] - ETA: 0s - loss: 1.3244 - accuracy: 0.4920
Epoch 00003: saving model to model_weight.h5
448/448 [==============================] - 933s 2s/step - loss: 1.3244 - accuracy: 0.4920 - val_loss: 1.5212 - val_accuracy: 0.4455
Epoch 4/15
448/448 [==============================] - ETA: 0s - loss: 1.2437 - accuracy: 0.5265
Epoch 00004: saving model to model_weight.h5
448/448 [=============================

In [7]:
model_json = model.to_json()
with open ("model.json","w") as json_file:
    json_file.write(model_json)